<a href="https://colab.research.google.com/github/kuba-cherryb/voice-anonymization-EN-PL/blob/main/voice_anonymization_PL_CMU_ARCTIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font size="4"><b>Technologia mowy 2024</b></font></center>
<br />
<center><font size="6"><b><u>Projekt 2: Anonimizacja mowy.</u></b></font></center>
<center>Jakub Czernecki, Wojciech Sabała, Bartosz Wąsik, 01/2025</center>

#Environment setup

##Python modules

In [1]:
import os

!git clone https://github.com/kuba-cherryb/voice-anonymization-EN-PL
%cd voice-anonymization-EN-PL
!pip install datasets
!pip install speechbrain
!pip install git+https://github.com/openai/whisper.git

from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset, Audio
import torch
import soundfile as sf
from IPython.display import Audio
from speechbrain.inference.speaker import EncoderClassifier
import torchaudio
import numpy as np
import warnings
import whisper
warnings.filterwarnings("ignore")

Cloning into 'voice-anonymization-EN-PL'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 1.91 MiB | 8.87 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/voice-anonymization-EN-PL
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take 

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


##Pretrained models

### Speech to Text - [Whisper](https://github.com/openai/whisper)

In [2]:
transcriptor = whisper.load_model("turbo")

100%|█████████████████████████████████████| 1.51G/1.51G [00:51<00:00, 31.4MiB/s]


###xvector extractor - [SpeechBrain Speaker Verification with xvector embeddings on Voxceleb](https://huggingface.co/speechbrain/spkrec-xvect-voxceleb)

In [3]:
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/hyperparams.yaml' -> '/content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/hyperparams.yaml'
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load_if_possible
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in pretrained_models/spkrec-xvect-voxceleb.
INFO:speechbrain.uti

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/embedding_model.ckpt' -> '/content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


...

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/mean_var_norm_emb.ckpt' -> '/content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/classifier.ckpt' -> '/content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached


label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-xvect-voxceleb/snapshots/56895a2df401be4150a159f3a1c653f00051d477/label_encoder.txt' -> '/content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvect-voxceleb/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /content/voice-anonymization-EN-PL/pretrained_models/spkrec-xvec

### Synthesizer - [SpeechT5](https://huggingface.co/blog/speecht5), [fine-tuned to Polish](kuba-cherryb/speecht5_tts_voxpopuli_pl_v4)

In [4]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("kuba-cherryb/speecht5_tts_voxpopuli_pl_v4")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

##xvector database - [CMU ARCTIC](https://huggingface.co/datasets/Matthijs/cmu-arctic-xvectors)

In [5]:
xvector_database = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
xvector_database = torch.tensor(xvector_database["xvector"])

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

#Analysis

## Speech to text

In [6]:
transcription = transcriptor.transcribe("resynthesis_pl.wav")
print(transcription["text"])

 Naczczo chleby i pyry z dżemem.


##Text cleanup

Tokenizer doesn't recognize foreign symbols and replaces them with `<unk>`. This creates a need to replace them with recognizable tokens. Additionally numbers are not recognized by the model, hence the replacement is necessary for them too.

In [7]:
replacements = [
    ('ó', 'œ'),
    ('ą', 'æ'),
    ('ć', 'ê'),
    ('ę', 'é'),
    ('ł', 'X'),
    ('ń', 'Q'),
    ('ś', 'V'),
    ('ź', 'q'),
    ('ż', 'v'),
    ('1', 'jeden'),
    ('2', 'dwa'),
    ('3', 'trzy'),
    ('4', 'cztery'),
    ('5', 'piéê'),
    ('6', 'szeVê'),
    ('7', 'siedem'),
    ('8', 'osiem'),
    ('9', 'dziewiéê'),
    ('0', 'zero'),
    ('%', 'procent'),
]


def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["text"] = inputs["text"].lower().replace(src, dst)
    return inputs

In [8]:
transcription = cleanup_text(transcription)
print(transcription["text"])

 naczczo chleby i pyry z dvemem.


##xvect_extraction

In [9]:
signal, fs = torchaudio.load('resynthesis_pl.wav')
extracted_xvector = classifier.encode_batch(signal[0]).squeeze(0)

#Anonymization

In [10]:
def anonymize(xvector):

  #computing euclidian distance from each database speaker
  #to determine the least similar speakers.
  euclid_dist = torch.cdist(xvector_database, xvector,p=2)
  ed_list = np.reshape(euclid_dist.tolist(), (-1))
  least_similar_idx = ed_list.argsort()[-3:][::-1]

  #averaging the least similar speakers to create anonymized
  axv = []

  for x in range(0,3):
    axv.append(xvector_database[least_similar_idx[x]])

  axv = np.mean(axv, axis=0)
  anon_xvector1 = torch.tensor(axv).unsqueeze(0)


  #max distance speaker
  maxid = np.argmax(ed_list)
  anxv3 = xvector_database[maxid]
  anon_xvector2 = torch.tensor(anxv3).unsqueeze(0)


  #determining random speakers to get a randomly distant, yet distant embedding
  randidx = np.random.randint(0, len(xvector_database),1)
  axv_rand = xvector_database[randidx]
  anon_xvector3 = torch.tensor(axv_rand)

  return anon_xvector1, anon_xvector2, anon_xvector3

In [17]:
anon_xvector1, anon_xvector2, anon_xvector3 = anonymize(xvector=extracted_xvector)

#Synthesis

##Original signal

In [12]:
Audio(signal, rate=16000)

##Model

###Original speech resynthesis

In [13]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], extracted_xvector, vocoder=vocoder)
Audio(speech, rate=16000)

###Distance anonimization

In [14]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector1, vocoder=vocoder)
Audio(speech, rate=16000)

###Max distance speaker

In [15]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector2, vocoder=vocoder)
Audio(speech, rate=16000)

###Speaker randomization

In [18]:
inputs = processor(text=transcription["text"], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], anon_xvector3, vocoder=vocoder)
Audio(speech, rate=16000)